In [2]:
import pandas as pd

# Chargement des données depuis un fichier CSV
df = pd.read_csv('../data/sbb-data.csv', delimiter=';')

# Séparation de la colonne Geopos en latitude et longitude
df[['latitude', 'longitude']] = df['Geopos'].str.split(',', expand=True)

# Conversion des nouvelles colonnes en type float pour un usage géospatial
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Vérification du résultat
print(df[['Geopos', 'latitude', 'longitude']].head())

# Sauvegarde des données dans un nouveau fichier CSV (optionnel)
df.to_csv('../data/sbb-data_latlong.csv', index=False)


                                 Geopos   latitude  longitude
0   47.5079223325029, 8.760389196167752  47.507922   8.760389
1  47.36660976671019, 8.548489088449456  47.366610   8.548489
2  47.40743350791071, 8.214805494441988  47.407434   8.214805
3   47.39917636823757, 8.44724506333892  47.399176   8.447245
4  47.42807508777434, 8.276363251209915  47.428075   8.276363
